# Objetivo

- During the COVID-19 crisis, several politicians used the "I'm following science" argument to justify their public policy decisions.

- My premise is that mayors who have worked throughout their lives in science and technology and related fields, known as STEM (Science, Technology, Engineering, and Math), would be the most likely to "follow the science" in their decision-making;

- I hypothesize that mayors with a "STEM background" would respond better to the challenges of the pandemic, reducing negative indicators, such as contagion and deaths;

- The following document seeks to create a suitable data frame for performing a data frame test of this hypothesis. Therefore, using Python, I searched through the TSE database for candidates elected in 2016 and 2020 who have worked with STEM throughout their lives (RAIS).

- Finally, using R and a Discontinuous Regression (RDD) model, I estimate the impact of electing a STEM candidate on cases and deaths caused by COVID. I found that having a STEM background did cause a lower rate of infections and deaths caused by COVID.

  -Primary Sources: RAIS, Database, Brazil IO

  -Secondary Sources: TSE, State Health Departments

# Configurando o ambiente

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import basedosdados as bd

# Procurando profissões STEM

In [2]:
cbos_stem = "1223 1236 1237 1413 1425 1426 2011 2012 2021 2030 2031 2032 2033 2034 2111 2112 2122 2123 2124 2131 2132 2133 2134 2140 2142 2143 2144 2145 2146 2147 2148 2149 2211 2212 2221 2222 2341 2342 2343 2344 2513 3001 3003 3011 3012 3111 3112 3115 3121 3122 3123 3161 3171 3172 3180 3181 3182 3183 3185 3186 3187 3201 3212 3253 3951"

- fonte: Machado et all 2021

In [3]:
cbos_stem = tuple(cbos_stem.split(" "))

In [4]:
query = """ SELECT * FROM `basedosdados.br_ibge_cbo_2002.sinonimo`
            WHERE SUBSTRING(cbo_2002, 0, 4) IN {} """.format(cbos_stem)

cbos = bd.read_sql(query, billing_project_id= "econometria-314719")

Downloading: 100%|███████████████████████████████████████████████████████████████| 536/536 [00:00<00:00, 1506.74rows/s]


In [5]:
cbos.tail(5)

,cbo_2002,sinonimo
531,321210,Técnico silvicultor
532,395105,Técnico de apoio em pesquisa e desenvolvimento...
533,395105,Técnico de apoio em pesquisa e desenvolvimento...
534,395105,Técnico de apoio em pesquisa e desenvolvimento...
535,395105,Técnico de apoio em pesquisa e desenvolvimento...


In [6]:
query = """ SELECT * FROM `basedosdados.br_ibge_cbo_2002.sinonimo`
            """

total_cbos = bd.read_sql(query, billing_project_id= "econometria-314719")

len(total_cbos)

Downloading: 100%|████████████████████████████████████████████████████████████| 7720/7720 [00:00<00:00, 11412.23rows/s]


7720

In [7]:
len(cbos)/len(total_cbos)

0.0694300518134715

- 7% das profissões do Brasil são, portanto, classificadas como STEM. Parece algo crível

# Procurando Candidatos STEM

## Procurando candidatos eleitos na BD+

In [8]:
query = """ SELECT
                resultados.ano,
                resultados.turno,
                resultados.sigla_uf,
                resultados.id_municipio,
                resultados.id_municipio_tse,
                resultados.cargo,
                resultados.sigla_partido,
                resultados.id_candidato_bd,
                resultados.resultado,
                resultados.votos,
                candidatos.cpf,
                candidatos.nome,
                candidatos.ocupacao,
                candidatos.idade,
                candidatos.genero,
                candidatos.instrucao            
            FROM `basedosdados.br_tse_eleicoes.candidatos` candidatos
            INNER JOIN `basedosdados.br_tse_eleicoes.resultados_candidato` resultados
            ON candidatos.id_candidato_bd = resultados.id_candidato_bd
            WHERE candidatos.cargo = 'prefeito'
            AND resultados.tipo_eleicao = 'eleicao ordinaria'
            AND resultados.ano = 2020
            AND candidatos.ano = 2020"""

eleitos = bd.read_sql(query, billing_project_id= "econometria-314719")

Downloading: 100%|███████████████████████████████████████████████████████████| 18131/18131 [00:05<00:00, 3198.76rows/s]


In [9]:
len(eleitos)

18131

In [10]:
eleitos = eleitos.drop_duplicates(subset = 'id_candidato_bd', keep="first")

In [11]:
len(eleitos)

18004

### Pegando primeiro e segundo colocados

In [12]:
eleitos[eleitos["turno"] == 2]["id_municipio"]

65       1302603
10535    1302603
Name: id_municipio, dtype: object

In [13]:
eleitos["teve_2_turno"] = eleitos["id_municipio"].apply(lambda x: 1 if x in list(eleitos[eleitos["turno"] == 2]["id_municipio"]) else 0)

In [14]:
len(eleitos[eleitos["teve_2_turno"] == 0])

18002

In [15]:
total_votos2 = pd.DataFrame(eleitos.groupby(["id_municipio_tse"]).votos.sum())

eleitos = eleitos.merge(total_votos2, how='inner', on='id_municipio_tse')

eleitos.rename(columns = {"votos_x": "votos", "votos_y": "total_votos"}, inplace=True)

eleitos["%votos"] = (eleitos.votos / eleitos.total_votos).round(5)

eleitos.shape

eleitos["id_municipio_tse"].unique().shape

(5555,)

In [16]:
grupo = pd.DataFrame(eleitos[eleitos["teve_2_turno"] == 0].groupby(["id_municipio_tse"])["%votos"].nlargest(2).reset_index())

In [17]:
grupo[grupo["id_municipio_tse"] == '10006']

,id_municipio_tse,level_1,%votos
0,10006,15444,0.52394
1,10006,15443,0.47606


In [18]:
dif_votos = grupo.groupby(["id_municipio_tse"]).diff(periods=-1)

In [19]:
dif_votos.drop(columns = "level_1", inplace = True)

In [20]:
dif_votos.rename(columns = {"%votos": "dif_votos"}, inplace = True)

grupo = grupo.join(dif_votos)

grupo = grupo.drop(columns = "level_1")

In [23]:
grupo = grupo.dropna()

grupo = grupo.drop(columns = "%votos")

eleitos = eleitos.merge(grupo, how = "inner", on = "id_municipio_tse")

In [26]:
cpfs_rdd = tuple(eleitos[eleitos["dif_votos"] <= 0.10].cpf.unique())

In [27]:
len(cpfs_rdd)

7516

## Criando função para procurar os cpfs eleitos na RAIS em todos os estados e em todos os anos antes de 2012

In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
#import basedosdados as bd
import pyodbc
import re
import time

sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_theme(style="darkgrid")

In [29]:
server = '****'

database = 'RAIS'

port= '****'

username = '****'

password = '****'

driver= '****'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

cursor = cnxn.cursor()

## Criando anos e estados de interesse

In [30]:
anos =[2003]
for i in range(2018-anos[0]):
    anos.append(anos[-1] + 1)

In [31]:
estados = list(eleitos.sigla_uf.unique())

## Criando função para vuscar e limpar a base na RAIS

In [32]:
df = pd.DataFrame()

In [34]:
def rais(ano,estado):
    global df
   
    if ano >= 2011:
        consulta_1 = """ SELECT
                    a.CPF cpf,
                    a.[Nome Trabalhador] nome,
                    a.[Município] id_municipio,
                    a.[Sexo Trabalhador] sexo,
                    a.[Raça Cor] raca,
                    a.[Idade] idade,
                    a.[CNPJ Raiz] cnpj_raiz,
                    a.[CBO Ocupação 2002] cbo_2002,
                    a.[CNAE 2 0 Classe] as cnae,
                    a.[Vl Remun Dezembro Nom] rem_dez,

                    a.[Data Admissão Declarada] data_adm,
                    a.[Tempo Emprego] tenure,
                    a.[Vínculo Ativo 31 12] vinc_ativ3112
                    FROM dbo.{}{}ID a
                    WHERE a.[Vínculo Ativo 31 12] != '0'
                    AND a.[Vínculo Ativo 31 12] != '0'
                    AND a.[Vínculo Ativo 31 12] != ' 0'
                    AND a.[Vínculo Ativo 31 12] != 0
                   """.format(estado,ano)
        
        df_provisorio = pd.read_sql(consulta_1,  cnxn)
        
        df_provisorio["Ano"] = ano
        df_provisorio["Estado"] = estado
        
        df_provisorio['cpf'] = df_provisorio['cpf'].str.strip()

        df_provisorio = df_provisorio.query("cpf in @cpfs_rdd")
    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 1') |
                      (df_provisorio['vinc_ativ3112'] == '1')  |
                      (df_provisorio['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1
    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 0') |
                      (df_provisorio['vinc_ativ3112'] == '0')  |
                      (df_provisorio['vinc_ativ3112'] == 'False'),'vinc_ativ3112'] = 0
           
        df = pd.concat([df, df_provisorio])
           
        print(f"foi criado um df de tamanho {len(df)}")

    elif ano >= 2006:
        
        consulta_1 = """ SELECT
                        a.CPF cpf,
                        a.[NOME] nome,
                        a.[MUNICIPIO] id_municipio,
                        a.[GENERO] sexo,
                        a.[RACA_COR] raca,
                        a.[DT NASCIMENT] dt_nasc,
                        a.[RADIC CNPJ] cnpj_raiz,
                        a.[OCUP 2002] cbo_2002,
                        a.[SB CLAS 20] as sbclas20,
                        a.[REM DEZEMBRO] rem_dez,

                        a.[DT ADMISSAO] data_adm,
                        a.[TEMP EMPR] tenure,
                        a.[EMP EM 31 12] vinc_ativ3112
                        FROM dbo.{}{}ID a
                        WHERE a.[EMP EM 31 12] != '0'
                        AND a.[EMP EM 31 12] != '0'
                        AND a.[EMP EM 31 12] != ' 0'
                        AND a.[EMP EM 31 12] != 0
                       """.format(estado,ano)
        df_provisorio = pd.read_sql(consulta_1,  cnxn)
        
        df_provisorio["Ano"] = ano
        df_provisorio["Estado"] = estado
        
        df_provisorio['cpf'] = df_provisorio['cpf'].str.strip()
        
        df_provisorio = df_provisorio.query("cpf in @cpfs_rdd")

    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 1') |
                      (df_provisorio['vinc_ativ3112'] == '1')  |
                      (df_provisorio['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1
    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 0') |
                      (df_provisorio['vinc_ativ3112'] == '0')  |
                      (df_provisorio['vinc_ativ3112'] == 'False'),'vinc_ativ3112'] = 0
           
        df = pd.concat([df, df_provisorio])
           
        print(f"foi criado um df de tamanho {len(df)}")
        
        
    else:
        
        consulta_1 = """ SELECT
                        a.CPF cpf,
                        a.[NOME] nome,
                        a.[MUNICIPIO] id_municipio,
                        a.[SEXO] sexo,
                        a.[RACA_COR] raca,
                        a.[DT NASCIMENT] dt_nasc,
                        a.[RADIC CNPJ] cnpj_raiz,
                        a.[OCUP 2002] cbo_2002,
                        a.[CLAS CNAE 95] as cnae_95,
                        a.[REM DEZEMBRO] rem_dez,

                        a.[DT ADMISSAO] data_adm,
                        a.[TEMP EMPR] tenure,
                        a.[EMP EM 31 12] vinc_ativ3112
                        FROM dbo.{}{}ID a
                        WHERE a.[EMP EM 31 12] != '0'
                        AND a.[EMP EM 31 12] != '0'
                        AND a.[EMP EM 31 12] != ' 0'
                        AND a.[EMP EM 31 12] != 0
                       """.format(estado,ano)
        df_provisorio = pd.read_sql(consulta_1,  cnxn)
        
        df_provisorio["Ano"] = ano
        df_provisorio["Estado"] = estado
        
        df_provisorio['cpf'] = df_provisorio['cpf'].str.strip()
        
        df_provisorio = df_provisorio.query("cpf in @cpfs_rdd")

    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 1') |
                      (df_provisorio['vinc_ativ3112'] == '1')  |
                      (df_provisorio['vinc_ativ3112'] == 'True'), 'vinc_ativ3112'] = 1
    
        df_provisorio.loc[(df_provisorio['vinc_ativ3112'] == ' 0') |
                      (df_provisorio['vinc_ativ3112'] == '0')  |
                      (df_provisorio['vinc_ativ3112'] == 'False'),'vinc_ativ3112'] = 0
           
        df = pd.concat([df, df_provisorio])
           
        print(f"foi criado um df de tamanho {len(df)}")

In [ ]:
start_time = time.time()

for x in estados:
    for i in anos:
        rais(i,x)
    
print("My program took", time.time() - start_time, "to run")

foi criado um df de tamanho 67
foi criado um df de tamanho 314


In [ ]:
df.info()

In [ ]:
df.head().T

In [ ]:
df['cbo_2002'] = df['cbo_2002'].str.extract('(\d+)').astype(str)

In [ ]:
tabela = pd.DataFrame(df.groupby('cbo_2002')['cbo_2002'].count().nlargest(20))

In [ ]:
tabela["cbos"] = tabela.index

In [ ]:
top_20_cbos = tuple(tabela.cbos)

## Procurando cbos na BD+

In [ ]:
import basedosdados as bd

query = """ SELECT * FROM `basedosdados.br_ibge_cbo_2002.sinonimo`
            WHERE cbo_2002 IN {} """.format(top_20_cbos)

tabela_top_20_cbos = bd.read_sql(query, billing_project_id= "econometria-314719")

In [ ]:
tabela_top_20_cbos

# Gerando variável de tempo trabalhado em profissão STEM "stem_tenure"

In [ ]:
lista = list(df["tenure"])

In [ ]:
coluna = []

for i in lista:
    if ',' in str(i).lower():  
        coluna.append(i.replace(',', '.'))
    else:
        coluna.append(i)

In [ ]:
df["teste"] = coluna

In [ ]:
df["teste2"] = df['teste'].astype(float)

In [ ]:
df["teste2"] = df['teste2'].astype(int)

In [ ]:
df_tenure = df.groupby(['cpf','cbo_2002'])['teste2'].sum().reset_index()

In [ ]:
df_tenure["stem_job"] = df_tenure["cbo_2002"].apply(lambda x: 1 if x[0:4] in cbos_stem else 0)

In [ ]:
df_tenure.query("stem_job == 1").head(10).T

In [ ]:
len(df_tenure.query("stem_job == 1").cpf.unique())

In [ ]:
len(df_tenure.query("stem_job == 0").cpf.unique())

In [ ]:
df_tenure_2 = df_tenure.groupby(['cpf','stem_job'])['teste2'].sum().reset_index()

In [ ]:
df_tenure_2 = df_tenure_2.rename(columns = {"teste2": "tenure"})

In [ ]:
df_tenure_2 = df_tenure_2.query("stem_job == 1")

# Juntar bases df_tenure_2 com df

In [ ]:
df = eleitos.merge(df_tenure_2, how = "left", on = "cpf")

In [ ]:
df["stem_job"].fillna(0, inplace=True)

In [ ]:
df["tenure"].fillna(0, inplace = True)

In [ ]:
df.sort_values(by = "id_municipio")

In [ ]:
(df[df["%votos"] == df["%votos"].max()])

# Mantendo só os 2 primeiros colocados na base

In [ ]:
df['max_votos'] = df.query("resultado != 'eleito'").groupby(['id_municipio_tse'])['%votos'].transform(max)

In [ ]:
lista = list(df[df['max_votos'] == df["%votos"]].id_candidato_bd)

In [ ]:
df["2_lugar"] = df["id_candidato_bd"].apply(lambda x: 1 if x in lista else 0)

In [ ]:
df["rdd"] = df["id_candidato_bd"].apply(lambda x: 1 if x in list(df[(df["resultado"] == 'eleito') | (df["2_lugar"] == 1)].id_candidato_bd) else 0)

In [ ]:
df.to_csv('Dados/Output/220531_base_rdd_prefeitos.csv', sep = ";", index = False, encoding = "utf-8-sig")

# Criar variável RDD se 1 colocado ou 2 colado são STEM

In [ ]:
df = pd.read_csv("Dados/Output/220531_base_rdd_prefeitos.csv", sep = ";")

In [ ]:
df.rename(columns = {"rdd": "1_ou_2_lugar"}, inplace = True)

In [ ]:
df = df[df["1_ou_2_lugar"] == 1]

In [ ]:
teste = pd.DataFrame(df.groupby(["id_municipio_tse"]).stem_job.sum())

In [ ]:
teste.rename(columns = {"stem_job": "rdd"}, inplace = True)

In [ ]:
df = df.merge(teste, how = "inner", on = "id_municipio_tse")

In [ ]:
df.query("rdd == 1").shape

In [ ]:
df.query("resultado == 'eleito' & rdd == 1 & stem_job == 1").shape

In [ ]:
df["rdd_stem_won"] = df.cpf.apply(lambda x: 1 if x in df.query("resultado == 'eleito' & rdd == 1 & stem_job == 1").cpf.unique() else 0)

In [ ]:
len(df[df["rdd_stem_won"] == 1])

In [ ]:
df.to_csv('Dados/Output/220531_base_rdd_covid_stem.csv', sep = ";", index = False, encoding = "utf-8-sig")

# Pegando óbitos por COVID

In [92]:
df_obitos = pd.read_csv("RAIS/Dados/Input/caso.csv")

fonte: https://brasil.io/dataset/covid19/caso/?search=&date=2022-01-01&state=&city=&place_type=city&is_last=&city_ibge_code=&order_for_place=

In [93]:
df_obitos = df_obitos.query("place_type == 'city'")

In [94]:
df_obitos.head()

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
734,2021-12-14,AP,Amapá,city,1424,13,586,True,9109.0,9187.0,1600105.0,15500.16327,0.0091
735,2021-12-13,AP,Amapá,city,1424,13,585,False,9109.0,9187.0,1600105.0,15500.16327,0.0091
736,2021-12-12,AP,Amapá,city,1424,13,584,False,9109.0,9187.0,1600105.0,15500.16327,0.0091
737,2021-12-11,AP,Amapá,city,1424,13,583,False,9109.0,9187.0,1600105.0,15500.16327,0.0091
738,2021-12-10,AP,Amapá,city,1424,13,582,False,9109.0,9187.0,1600105.0,15500.16327,0.0091


In [95]:
df_obitos = df_obitos[(df_obitos["date"] >= "2021-06-19") & (df_obitos["date"] <= "2021-07-01")]

In [96]:
df_obitos['mais_recente'] = df_obitos.groupby(['city'])['date'].transform(max)

In [97]:
df_obitos['mais_recente'] = (df_obitos["date"] == df_obitos["mais_recente"])

In [99]:
df_obitos = df_obitos[df_obitos['mais_recente'] == 1]

In [102]:
df_obitos.rename(columns = {"city_ibge_code": "id_municipio"}, inplace = True)

# Mesclando as bases

In [103]:
df = pd.read_csv("RAIS/Dados/Output/220531_base_rdd_covid_stem.csv", sep = ";")

In [104]:
df = df.merge(df_obitos, how = "left", on = "id_municipio")

In [109]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10583 entries, 0 to 10582
Data columns (total 40 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ano                             10583 non-null  int64  
 1   turno                           10583 non-null  int64  
 2   sigla_uf                        10583 non-null  object 
 3   id_municipio                    10583 non-null  int64  
 4   id_municipio_tse                10583 non-null  int64  
 5   cargo                           10583 non-null  object 
 6   sigla_partido                   10583 non-null  object 
 7   id_candidato_bd                 10583 non-null  int64  
 8   resultado                       10583 non-null  object 
 9   votos                           10583 non-null  int64  
 10  cpf                             10583 non-null  int64  
 11  nome                            10583 non-null  object 
 12  ocupacao                        

# Salvando

In [111]:
df.to_csv('RAIS/Dados/Output/220601_base_rdd_covid_stem.csv', sep = ";", index = False, encoding = "utf-8-sig")